In [1]:
from sqlalchemy import create_engine
import pandas as pd
import sqlalchemy
import dtale

%config IPCompleter.greedy=True

core_address = 'analysts-dwh.cabify-data.com'
core_port = '5439'

f = open("/Users/santiagocanas/Downloads/redshift.txt", mode = "r")
file = f.readlines()

core_username = str(file[0].strip())
core_password = str(file[1].strip())
core_dbname = 'dwh'

dwh_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'.format(
                        username = core_username,
                        password = core_password,
                        ipaddress = core_address,
                        port = core_port,
                        dbname = core_dbname))

dwh_cnx = create_engine(dwh_str)

In [ ]:
sql = '''
select
    jt.fk_journey_id,
    sum(js.qt_price_local) as price_supplements
from
    datawarehouse.ops_fac_journeytotals_supplements js
        inner join
    datawarehouse.ops_fac_journeytotals jt on jt.sk_journeytotals = js.fk_journeytotals
where
    exists (select
                *
            from
                datawarehouse.fin_dim_sale_entries se
                    inner join
                datawarehouse.fin_dim_sale s on se.fk_sale = s.sk_sale
            where
                jt.fk_journey_id = se.fk_journey_id
                and s.cd_code IN ('ESPV21GS039156')
                and s.dt_date >= date_trunc('month', current_date - interval '2 month')
                and s.ds_sale_state = 'closed'
                and s.in_delete is null
    )
group by 
    1
'''
data = pd.read_sql_query(sql, dwh_cnx)
data

In [ ]:
sql = '''
	select
		s.sk_sale,
		mtm.id_journey as journey_id,
		mtm.qt_real_distance as moving,
		mtm.ds_pm_rider as payment_method_rider,
		mtm.ds_platform as platform,
		jt.qt_cost_total_local as cost,
		mtm.ds_region_name as region_name,
		mtm.ds_district_start as ds_district,
		mtm.ds_neighborhood_start as ds_neighborhood,
		mtm.ds_product_slug as slug,
		mtm.id_company_id as id_company,
		mtm.ds_company_name as company_name,
		mtm.id_driver_id as id_driver,
		rc.cd_iata_code as region_code
--		js.price_supplements
	from
		datawarehouse.fin_dim_sale s
			inner join
		datawarehouse.fin_dim_sale_entries se on s.sk_sale = se.fk_sale
			left join
		datawarehouse.ops_mtm_journey mtm on se.fk_journey_id = mtm.sk_journey
			left join
		datawarehouse.ops_fac_journeytotals jt on mtm.sk_journey = jt.fk_journey_id
			left join
		datawarehouse.ops_dim_region_code rc on rc.fk_region_id = mtm.fk_region_id
--			left join
--		journey_supplements js on se.fk_journey_id = js.fk_journey_id
	where
		s.dt_date >= date_trunc('month', current_date - interval '2 month')
        and s.cd_code IN ('ESPV21GS039156')
		and s.ds_sale_state = 'closed'
		and s.in_delete is null
'''
dat1 = pd.read_sql_query(sql, dwh_cnx)
dat1

In [15]:
sql = '''
select
	s.cd_code as sale_code,
    s.fk_client_id
from
	datawarehouse.fin_dim_sale s
		inner join
	datawarehouse.fin_dim_sale_entries se on s.sk_sale = se.fk_sale
		inner join
	datawarehouse.fin_dim_sale sr on s.fk_rectify_sale_id = sr.sk_sale
		inner join
	datawarehouse.fin_dim_invoice i on s.fk_invoice_id = i.sk_invoice
		inner join
	datawarehouse.ops_dim_client c on s.fk_client_id = c.sk_client
where
	s.dt_date >= date_trunc('month', current_date - interval '2 month')
    and s.cd_code IN ('ESPV21GS039156')
	and s.ds_sale_state = 'closed'
	and s.in_delete is null
'''
dat2 = pd.read_sql_query(sql, dwh_cnx)
dat2

,sale_code,fk_client_id
0,ESPV21GS039156,38715851


In [11]:
sql = '''
select 
    sk_client,
    id_client
from
    datawarehouse.ops_dim_client c
where 
    c.sk_client = 38715851
'''
cli = pd.read_sql_query(sql, dwh_cnx)
cli

,sk_client,id_client


In [12]:
sql = '''
select 
    *
from
    datawarehouse.ops_dim_user
where
    sk_user = 38715851
'''
user = pd.read_sql_query(sql, dwh_cnx)
user

,sk_user,id_user,fk_active_payment_method_id,in_activated,fk_admin_agency_id,fk_active_region_id,fk_agency_id,cd_auth_token,dt_authorization_service_date,ds_avatar_filename,...,ds_loyaltyprogram_card_number_encrypt,tm_mobile_verified_utc_at,dt_mobile_verified_utc_at,tm_at_movo_tos_utc_at,dt_at_movo_tos_utc_at,ad_inserted_at,tm_at_aragon_tos_utc_at,dt_at_aragon_tos_utc_at,tm_at_wible_tos_utc_at,dt_at_wible_tos_utc_at
0,38715851,b1df52defd1343dbae23c68cde5ca200,-1,None,-1,-1,-1,Y6qy7lx1ZD2698A0f2rnbg,None,None,...,None,2021-02-06 23:49:18.117,2021-02-06,None,None,2021-02-08 02:35:06.750998,2021-02-06 23:49:19,2021-02-06,None,None


In [13]:
list(user.columns)

['sk_user',
 'id_user',
 'fk_active_payment_method_id',
 'in_activated',
 'fk_admin_agency_id',
 'fk_active_region_id',
 'fk_agency_id',
 'cd_auth_token',
 'dt_authorization_service_date',
 'ds_avatar_filename',
 'ds_avatar_url',
 'dt_birthday',
 'fk_blacklist_detail_id',
 'fk_campaign_click_id',
 'fk_campaign_source_id',
 'in_can_manage',
 'in_can_order_for_others',
 'fk_client_id',
 'fk_company_id',
 'ds_company_name',
 'cd_country',
 'tm_created_utc_at',
 'dt_created_utc_at',
 'cd_default_charge_code',
 'in_disable_user_events',
 'in_disabled',
 'in_da_approved',
 'fk_da_funnel_id',
 'ds_da_funnel_name',
 'id_da_id',
 'ds_driver_course_status',
 'ds_driver_license',
 'ds_email',
 'in_email_bounced',
 'ds_fleet_role',
 'ds_full_name',
 'ds_gender',
 'ds_general_register_number',
 'cd_employee_code',
 'tm_info_invitations_sent_utc_at',
 'dt_info_invitations_sent_utc_at',
 'fk_group_id',
 'cd_invitation_code',
 'cd_language',
 'in_loc',
 'ds_linkedin_url',
 'cd_locale',
 'in_marketing_

In [21]:
agg = pd.read_excel('/Users/santiagocanas/Downloads/query_agg.xlsx')

In [22]:
agg.head()

,issuer,actionSource,steps,times
0,BANCO SANTANDER S.A.,ACTION_SOURCE_SCA_CHECKER,"IDENTIFY_SHOPPER,AUTHENTICATION_FINISHED,AUTHO...",2786
1,"BANKIA, S.A.",ACTION_SOURCE_SCA_CHECKER,"IDENTIFY_SHOPPER,AUTHENTICATION_FINISHED,AUTHO...",2322
2,BANKIA,ACTION_SOURCE_SCA_CHECKER,"IDENTIFY_SHOPPER,AUTHENTICATION_FINISHED,AUTHO...",1975
3,"BANCO SANTANDER, S.A.",ACTION_SOURCE_SCA_CHECKER,"IDENTIFY_SHOPPER,AUTHENTICATION_FINISHED,AUTHO...",1535
4,NaN,ACTION_SOURCE_SCA_CHECKER,"IDENTIFY_SHOPPER,AUTHENTICATION_FINISHED,AUTHO...",1525


In [23]:
query = pd.read_excel('/Users/santiagocanas/Downloads/query.xlsx')

In [27]:
query.groupby(['issuer','actionSource','steps']).agg({'id':'nunique'}).sort_values(by = 'id', ascending= False)

,,,id
issuer,actionSource,steps,
BANCO SANTANDER S.A.,ACTION_SOURCE_SCA_CHECKER,"IDENTIFY_SHOPPER,AUTHENTICATION_FINISHED,AUTHORISED",2788
"BANKIA, S.A.",ACTION_SOURCE_SCA_CHECKER,"IDENTIFY_SHOPPER,AUTHENTICATION_FINISHED,AUTHORISED",2325
BANKIA,ACTION_SOURCE_SCA_CHECKER,"IDENTIFY_SHOPPER,AUTHENTICATION_FINISHED,AUTHORISED",1978
"BANCO SANTANDER, S.A.",ACTION_SOURCE_SCA_CHECKER,"IDENTIFY_SHOPPER,AUTHENTICATION_FINISHED,AUTHORISED",1536
"CAIXABANK, S.A.",ACTION_SOURCE_SCA_CHECKER,"IDENTIFY_SHOPPER,AUTHENTICATION_FINISHED,AUTHORISED",1262
...,...,...,...
"EURO 6000, S.A.",ACTION_SOURCE_SCA_CHECKER,"IDENTIFY_SHOPPER,AUTHENTICATION_FINISHED,AUTHORISED",1
DNB NOR BANK ASA,ACTION_SOURCE_SCA_CHECKER,"IDENTIFY_SHOPPER,AUTHENTICATION_FINISHED,AUTHORISED",1
DNB BANK ASA,ACTION_SOURCE_SCA_CHECKER,"IDENTIFY_SHOPPER,AUTHENTICATION_FINISHED,AUTHORISED",1


In [29]:
sql = '''
SELECT 
	p.fk_client_id,
	COUNT(DISTINCT p.id_payment)
from 
	datawarehouse.fin_fac_payment p
where 
	p.ds_state = 'canceled'
	and p.tm_created_at >= current_date - interval'1 month'
group by 
	1
having 
	COUNT(DISTINCT p.id_payment) > 1 '''
cli = pd.read_sql_query(sql, dwh_cnx)
cli

,fk_client_id,count
0,22860038,7
1,716879,2
2,38272837,2
3,11607015,3
4,28677223,8
...,...,...
271411,21154406,2
271412,16007655,2
271413,9132500,2
271414,34194823,2
